In [1]:
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

#### Hyperparameter

In [2]:
EPOCHS = 1

#### Dense Unit 구현

In [10]:
class DenseUnit(tf.keras.Model):
    def __init__(self, filter_out, kernel_size):
        super(DenseUnit, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filter_out, kernel_size, padding="same")
        self.concat = tf.keras.layers.Concatenate()
        
    def call(self, x, training=False, mask=None):
        # x : (Batch, H, W, Ch_in)
        h = self.bn(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv(h)
        # h : (Batch, H, W, filter_output)
        return self.concat([x, h]) # (Batch, H, W, (Ch_in + filter_output))

#### DenseLayer 구현

In [11]:
class DenseLayer(tf.keras.Model):
    def __init__(self, num_unit, growth_rate, kernel_size):
        super(DenseLayer, self).__init__()
        self.sequence = list()
        for idx in range(num_unit):
            self.sequence.append(DenseUnit(growth_rate, kernel_size))
        
    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

#### Transition Layer 구현

In [12]:
class TransitionLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(TransitionLayer, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding="same")
        self.pool = tf.keras.layers.MaxPool2D()
        
    def call(self, x, training=False, mask=None):
        x = self.conv(x)
        return self.pool(x)

#### Model

In [18]:
class DenseNet(tf.keras.Model):
    def __init__(self):
        super(DenseNet, self).__init__()
        # 28x28x8
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu")
        # 28x28x16
        self.dl1 = DenseLayer(2, 4, (3, 3))
        # 14x14x16
        self.tr1 = TransitionLayer(16, (3, 3))
        
        # 14x14x32
        self.dl2 = DenseLayer(2, 8, (3, 3))
        # 7x7x32
        self.tr2 = TransitionLayer(32, (3, 3))
        
        # 7x7x64
        self.dl3 = DenseLayer(2, 16, (3, 3))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation="relu")
        self.dense2 = tf.keras.layers.Dense(10, activation="softmax")
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.dl1(x, training=training)
        x = self.tr1(x)
        
        x = self.dl2(x, training=training)
        x = self.tr2(x)
        
        x = self.dl3(x, training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

#### Train, Test loop

In [19]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

#### Data set

In [20]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [21]:
# Create model
model = DenseNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

#### Training

In [23]:
for epoch in range(EPOCHS):
    for images, labels in tqdm(train_ds):
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    for test_images, test_labels in tqdm(test_ds):
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}. Test Accuracy: {}"
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))


Epoch 1, Loss: 0.05584270879626274, Accuracy: 98.36500549316406, Test Loss: 0.058833472430706024. Test Accuracy: 98.22000122070312
